In [1]:
import dac
import torch
from torchaudio.transforms import Resample
from datasets import load_dataset
from audiotools import AudioSignal
from IPython.display import Audio

2024-03-23 16:06:20.612652: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 16:06:20.612703: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 16:06:20.612744: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 16:06:20.621560: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset = load_dataset("danjacobellis/audio_har",split='semi_natural')
dataset = dataset.with_format('torch')
fs_orig = dataset[0]['path']['sampling_rate']
fs = 24000
resampler = Resample(fs_orig,fs)

In [3]:
model_path = dac.utils.download(model_type="24khz")
model = dac.DAC.load(model_path)
model.to('cuda');

/home/server/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
def audio_to_code(sample):
    with torch.no_grad():
        signal = AudioSignal(sample['path']['array'],sample_rate=fs)
        signal.to(model.device)
        x = model.preprocess(signal.audio_data, signal.sample_rate)
        z, codes, latents, _, _ = model.encode(x)
        sample['codes'] = codes.detach().cpu()
        return sample

In [5]:
dataset = dataset.map(audio_to_code)

Map:   0%|          | 0/669 [00:00<?, ? examples/s]